<a href="https://colab.research.google.com/github/Tarak5556N/ATM-interface/blob/main/ATM_Project_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import datetime

class ATM:
    def __init__(self):
        self.users = {}  # To store user details as {username: {'pin': pin, 'balance': balance, 'transactions': []}}
        self.current_user = None
        self.create_selection_screen()

    def create_selection_screen(self):
        clear_output()
        self.display_logo()

        selection_label = widgets.Label(value="Select an option:")
        login_button = widgets.Button(description='Login', button_style='info')
        signup_button = widgets.Button(description='Signup', button_style='success')

        login_button.on_click(lambda _: self.show_login_form())
        signup_button.on_click(lambda _: self.show_signup_form())

        display(widgets.VBox([selection_label, login_button, signup_button], layout=widgets.Layout(align_items='center')))

    def show_login_form(self):
        clear_output()
        self.display_logo()

        self.username_input = widgets.Text(description='Username:', style={'description_width': 'initial'})
        self.pin_input = widgets.Password(description='PIN:', style={'description_width': 'initial'})
        login_button = widgets.Button(description='Login', button_style='success')
        login_button.on_click(self.authenticate)
        back_button = widgets.Button(description='Back', button_style='primary')
        back_button.on_click(lambda _: self.create_selection_screen())

        display(widgets.VBox([self.username_input, self.pin_input, login_button, back_button], layout=widgets.Layout(align_items='center')))

    def show_signup_form(self):
        clear_output()
        self.display_logo()

        self.username_input = widgets.Text(description='Username:', style={'description_width': 'initial'})
        self.pin_input = widgets.Password(description='PIN:', style={'description_width': 'initial'})
        signup_button = widgets.Button(description='Signup', button_style='success')
        signup_button.on_click(self.register_user)
        back_button = widgets.Button(description='Back', button_style='primary')
        back_button.on_click(lambda _: self.create_selection_screen())

        display(widgets.VBox([self.username_input, self.pin_input, signup_button, back_button], layout=widgets.Layout(align_items='center')))

    def display_logo(self):
        logo_html = """
        <div style="text-align: center; margin: 20px;">
            <img src="https://th.bing.com/th/id/OIP.aYxhaiB3ZwX3ZOnMZAUPiwHaC-?rs=1&pid=ImgDetMain" width="100">
        </div>
        """
        display(HTML(logo_html))

    def register_user(self, b):
        username = self.username_input.value.strip()
        pin = self.pin_input.value.strip()
        if username and pin:
            if username in self.users:
                display(HTML("<p style='color: red; text-align: center;'>Username already exists. Please choose another.</p>"))
                self.show_signup_form()
            else:
                self.users[username] = {'pin': pin, 'balance': 0.0, 'transactions': []}
                display(HTML("<p style='color: green; text-align: center;'>Registration successful! Please log in.</p>"))
                self.show_login_form()
        else:
            display(HTML("<p style='color: red; text-align: center;'>Username and PIN cannot be empty.</p>"))
            self.show_signup_form()

    def authenticate(self, b):
        username = self.username_input.value.strip()
        pin = self.pin_input.value.strip()
        if username in self.users and self.users[username]['pin'] == pin:
            self.current_user = username
            self.create_main_menu()
        else:
            display(HTML("<p style='color: red; text-align: center;'>Incorrect username or PIN. Please try again.</p>"))
            self.show_login_form()

    def create_main_menu(self):
        clear_output()
        self.display_logo()

        menu_html = "<h2 style='text-align: center; color: #4CAF50; margin: 20px;'>Welcome to BrainWave Matrix ATM, {}</h2>".format(self.current_user)
        display(HTML(menu_html))

        # Create menu buttons and their animations
        self.balance_button = self.create_animated_button('Check Balance', 'info')
        self.deposit_button = self.create_animated_button('Deposit', 'info')
        self.withdraw_button = self.create_animated_button('Withdraw', 'info')
        self.history_button = self.create_animated_button('Transaction History', 'info')
        self.exit_button = self.create_animated_button('Exit', 'danger')

        # Assign actions to buttons
        self.balance_button.on_click(self.check_balance)
        self.deposit_button.on_click(self.deposit)
        self.withdraw_button.on_click(self.withdraw)
        self.history_button.on_click(self.show_transaction_history)
        self.exit_button.on_click(self.exit)

        # Display buttons in a vertical layout
        button_box = widgets.VBox([
            self.balance_button,
            self.deposit_button,
            self.withdraw_button,
            self.history_button,
            self.exit_button
        ], layout=widgets.Layout(align_items='center', margin='20px', grid_gap='10px'))

        display(button_box)

    def create_animated_button(self, description, style):
        button = widgets.Button(description=description, button_style=style, layout=widgets.Layout(width='200px', margin='10px'))
        button.add_class('animated-button')
        return button

    def check_balance(self, b):
        clear_output()
        self.display_logo()
        balance = self.users[self.current_user]['balance']
        display(HTML("""
            <div style='text-align: center;'>
                <p style='font-size: 20px; color: #4CAF50;'>Hello, {}! Your current balance is: ${:.2f}</p>
            </div>
        """.format(self.current_user, balance)))
        self.create_back_button()

    def create_back_button(self):
        self.back_button = self.create_animated_button('Back', 'primary')
        self.back_button.on_click(lambda _: self.create_main_menu())
        display(widgets.VBox([self.back_button], layout=widgets.Layout(align_items='center')))

    def deposit(self, b):
        clear_output()
        self.display_logo()
        self.amount_input = widgets.FloatText(description='Amount:')
        self.submit_deposit = self.create_animated_button('Submit', 'success')
        self.submit_deposit.on_click(self.process_deposit)
        display(widgets.VBox([self.amount_input, self.submit_deposit], layout=widgets.Layout(align_items='center', margin='20px')))

    def process_deposit(self, b):
        amount = self.amount_input.value
        if amount > 0:
            self.users[self.current_user]['balance'] += amount
            self.record_transaction("Deposit", amount)
            display(HTML("""
                <div style='text-align: center;'>
                    <p style='font-size: 20px; color: #4CAF50;'>${:.2f} deposited successfully.</p>
                </div>
            """.format(amount)))
        else:
            display(HTML("<p style='color: red; text-align: center;'>Invalid deposit amount.</p>"))
        self.create_back_button()

    def withdraw(self, b):
        clear_output()
        self.display_logo()
        self.amount_input = widgets.FloatText(description='Amount:')
        self.submit_withdraw = self.create_animated_button('Submit', 'success')
        self.submit_withdraw.on_click(self.process_withdraw)
        display(widgets.VBox([self.amount_input, self.submit_withdraw], layout=widgets.Layout(align_items='center', margin='20px')))

    def process_withdraw(self, b):
        amount = self.amount_input.value
        if 0 < amount <= self.users[self.current_user]['balance']:
            self.users[self.current_user]['balance'] -= amount
            self.record_transaction("Withdraw", amount)
            display(HTML("""
                <div style='text-align: center;'>
                    <p style='font-size: 20px; color: #4CAF50;'>${:.2f} withdrawn successfully.</p>
                </div>
            """.format(amount)))
        else:
            display(HTML("<p style='color: red; text-align: center;'>Invalid withdrawal amount or insufficient balance.</p>"))
        self.create_back_button()

    def show_transaction_history(self, b):
        clear_output()
        self.display_logo()
        transactions = self.users[self.current_user]['transactions']
        if transactions:
            history_html = """
            <h2 style='text-align: center; color: #4CAF50;'>Transaction History:</h2>
            <table style='margin-left: auto; margin-right: auto; border-collapse: collapse;'>
                <tr style='background-color: #4CAF50; color: white;'>
                    <th style='padding: 10px; border: 1px solid #ddd;'>Date</th>
                    <th style='padding: 10px; border: 1px solid #ddd;'>Type</th>
                    <th style='padding: 10px; border: 1px solid #ddd;'>Amount</th>
                </tr>
            """
            for transaction in transactions:
                history_html += "<tr><td style='padding: 10px; border: 1px solid #ddd;'>{}</td><td style='padding: 10px; border: 1px solid #ddd;'>{}</td><td style='padding: 10px; border: 1px solid #ddd;'>${:.2f}</td></tr>".format(transaction['date'], transaction['type'], transaction['amount'])
            history_html += "</table>"
            display(HTML(history_html))
        else:
            display(HTML("<p style='text-align: center; color: #4CAF50;'>No transactions found.</p>"))
        self.create_back_button()

    def record_transaction(self, transaction_type, amount):
        now = datetime.datetime.now()
        transaction = {
            "type": transaction_type,
            "amount": amount,
            "date": now.strftime("%Y-%m-%d %H:%M:%S")
        }
        self.users[self.current_user]['transactions'].append(transaction)

    def exit(self, b):
        clear_output()
        self.display_logo()
        display(HTML("""
            <div style='text-align: center;'>
                <p style='font-size: 20px; color: #4CAF50;'>Thank you for using the ATM. Goodbye!</p>
            </div>
        """))
        self.create_selection_screen()

if __name__ == "__main__":
    atm = ATM()
